In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import keras_tuner
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D
from tensorflow import keras
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from methods_audio import data_handling
from methods_audio import data_augmentation
from methods_audio import denoising 
from methods_audio import model_performance_training
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# 1. Get data (file names)

In [ ]:
data = data_handling.get_data()

# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [ ]:
data = data.map(data_handling.read_in_data) 

# 3. Get input for model training 

In [ ]:
samples, labels = data_handling.extract_samples_labels(data)

# 4. Split data into train and validation sets

In [ ]:
validation_set_size = 0.30
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size= validation_set_size, random_state=123)

# 5. Transform data to spectograms

In [ ]:
type_transformation = 'mfcc'
x_train = data_handling.transform_data(x_train, type_transformation)
x_valid = data_handling.transform_data(x_valid, type_transformation)

## 6. Build model with hyperparameter tunning 
https://keras.io/guides/keras_tuner/getting_started/ <br>
https://www.youtube.com/watch?v=6Nf1x7qThR8&ab_channel=GregHogg 

In [ ]:
def build_model(hp):
    input = (624, 129,1)
    model = keras.Sequential()
    
    # Add input layer 
    #matching samples.shape
    model.add(
        Conv2D(
            filters= hp.Int("conv_filters_0", min_value=16, max_value=128, step=16), 
            activation= hp.Choice("conv_activation_0", ["relu", "tanh"]),
            kernel_size = (3,3), 
            input_shape=input
        )
    ) 
    model.add(MaxPool2D(pool_size= (2,2)))

    # Tune the number of Conv layers 
    for i in range(hp.Int("num_conv_layers", 1, 3)):
        model.add(
            Sequential([
                layers.Conv2D(
                    filters=hp.Int(f"conv_filters_{i}", min_value=16, max_value=128, step=16),
                    activation=hp.Choice(f"conv_activation_{i}", ["relu", "tanh"]),
                    kernel_size=(3,3),
                ), 
                layers.MaxPool2D(pool_size=(2,2)),
            ])
        )

    model.add(layers.Flatten())

    # Tune the number of Dense layers and Tune whether to use dropout layer
    for i in range(hp.Int("num_dense_layers", 1, 3)):
            model.add(
                Sequential([
                    layers.Dense(
                        # Tune number of units separately.
                        units=hp.Int(f"dense_units_{i}", min_value=50, max_value=600, step=50),
                        activation=hp.Choice(f"dense_activation_{i}", ["relu", "tanh"]),
                    ), 
                    layers.Dropout(
                        rate=hp.Float(f"dense_dropout_{i}", min_value = 0, max_value = 1)
                    )
                ]) 
            )

    model.add(
        layers.Dense(
        units=1, #because we have 2 classes 
        activation=hp.Choice("activatio_last_layer", ["softmax", "sigmoid"]), 
        )
    )

    # Define the optimizer learning rate as a hyperparameter.
    # sampling="log", the step is multiplied between samples.
    lr= hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=lr), 
        loss="BinaryCrossentropy", 
        metrics=["accuracy"],
    )
    
    return model

In [ ]:
build_model(keras_tuner.HyperParameters())

### Initialize tuner by specifying different arguments 

In [ ]:
tuner = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective= "val_accuracy", # we want maximize accuracy 
    overwrite=True,
    directory="param_optimization2",
    project_name="first_try",
)


In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor= 'val_loss', patience=5) 
# patience refers to number of epochs: if the val loss is not improving fter 5 ephocs, we stop it. 

### During the search, the model is called with different hyperparameters 

In [ ]:
tuner.search_space_summary()
# Default search space size: number of hyper parameters that we are tunning 

In [ ]:
epochs = 20
tuner.search(np.stack(x_train), np.stack(y_train), epochs= epochs, validation_data=(np.stack(x_valid), np.stack(y_valid)), callbacks=[stop_early]) #similar to fit 

In [ ]:
tuner.results_summary()

### After all of that we don't have a model yet but rather a set of hyper parameters. Let's query the results and create a model:

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
model = tuner.hypermodel.build(best_hps) #saving model 

In [ ]:
location = 'data/models/02'
model.save(location)

# 7. Evaluate model performance 

In [ ]:
# evaluate the model on a validation set
loss, accuracy = model.evaluate(np.stack(x_valid), np.stack(y_valid))

# print the evaluation results
print(f'Validation loss: {loss:.4f}')
print(f'Validation accuracy: {accuracy:.4f}')

In [ ]:
predictions = model.predict(np.stack(x_valid))


In [ ]:

y_pred = [1 if prediction > 0.5 else 0 for prediction in predictions]
accuracy = accuracy_score(np.stack(y_valid), y_pred)
accuracy



In [ ]:
precision = precision_score(np.stack(y_valid), y_pred)
recall = recall_score(np.stack(y_valid), y_pred)
f1 = f1_score(np.stack(y_valid), y_pred)

print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')